<a href="https://colab.research.google.com/github/nosark/ppo-lunar-lander-v2/blob/main/hugface_unit1_hands_on.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install dependencies and create a virtual screen to view agent results
---

In [ ]:
!apt install swig cmake


In [ ]:
!pip install -r https://raw.githubusercontent.com/huggingface/deep-rl-class/main/notebooks/unit1/requirements-unit1.txt

In [ ]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg
!apt install xvfb
!pip3 install pyvirtualdisplay



---



## Start Importing Packages
---

In [ ]:
import os

os.kill(os.getpid(), 9)

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [6]:
import gymnasium

from huggingface_sb3 import load_from_hub, package_to_hub
from huggingface_hub import (
    notebook_login,
)

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor



### Test Environment to understand the problem
---

In [ ]:
import gymnasium as gym

# Create enviroment for lunar lander
env = gym.make("LunarLander-v2")

# reset enviroment to start fresh
observation, info = env.reset()

for _ in range(20):
  # Take a random action
  action = env.action_space.sample()
  print("Action taken:", action)

  # Take action, and collect resulting: state, reward, terminated, truncated and info
  observation, reward, terminated, truncated, info = env.step(action)

  # If the game terminates (land and crash) or truncates(timeout)
if terminated or truncated:
  # Reset environment
  print("Environment is reset!")
  observation, info = env.reset()

env.close()

## Create Lunar Lander Enviroment
---

In [8]:
# Create the LunarLander enviroment
env = gym.make("LunarLander-v2")
env.reset()

print("---- OBSERVATION SPACE ---- \n")
print("Observation Space Shape", env.observation_space.shape)
# Get a random Observation
print("Sample Observation", env.observation_space.sample())

---- OBSERVATION SPACE ---- 

Observation Space Shape (8,)
Sample Observation [  9.071415   -43.454147    -0.06194903   0.36985514   0.7584448
   2.0781524    0.44606757   0.5876229 ]


### Observation Space Details
Observation Space Shape (8)

###### [HPC, VPC, HS, VS, Ang, AngSpeed, rightLegContacted, leftLegContacted]
---
- Horizontal Pad Coordinate(x)
- Vertical Pad Coordinate(y)
- Horizontal Speed(x)
- Vertical Speed(y)
- Angle
- Angular Speed
- If the left leg contact point has touched ground
- If the right leg contact point has touched ground

In [9]:
print("\n ---- ACTION SPACE ----\n")
print("Action Space Shape", env.action_space.n)
# Get random action space sample
print("Action Space Sample", env.action_space.sample())


 ---- ACTION SPACE ----

Action Space Shape 4
Action Space Sample 1


### Action Space Details
Action Space Shape (4)

[0, 1, 2, 3]
---
- Action 0 : Do nothing
- Action 1 : Fire left orientation engine
- Action 2 : Fire main engine
- Action 3 : Fire right orientation engine

#### Stacking 16 enviroments to improve diversity of events during training

In [10]:
env = make_vec_env("LunarLander-v2", n_envs=16)


In [ ]:
from collections.abc import Callable
env = Monitor(gym.make("LunarLander-v2"))

# Create the agent
model = PPO(
    policy="MlpPolicy",
    env=env,
    learning_rate=0.0008,
    n_steps=1280,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    clip_range=0.3,
    verbose=1,
)

model.learn(total_timesteps=1000000)

model_name="ppo-LunarHopefullyLander-v2"
model.save(model_name)


### Evaluate the model

In [14]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean reward={mean_reward:.2f} +/- {std_reward}")


mean reward=246.76 +/- 72.10850129133831


#### Post to Hugging Face

In [15]:
notebook_login()
!git config --global credential.helper store

In [ ]:
import gymnasium as gym
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.env_util import make_vec_env

from huggingface_sb3 import package_to_hub
env_id = "LunarLander-v2"
model_architecture = "PPO"
repo_id = "nosark/ppo-lunar-lander-v2"
commit_message= "Upload PPO LunarLander-v2 trained agent to HuggingFace"
# Create the evaluation env and set the render_mode="rgb_array"
eval_env = DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))])

# PLACE the package_to_hub function you've just filled here
package_to_hub(
    model=model,  # Our trained model
    model_name=model_name,  # The name of our trained model
    model_architecture=model_architecture,  # The model architecture we used: in our case PPO
    env_id=env_id,  # Name of the environment
    eval_env=eval_env,  # Evaluation Environment
    repo_id=repo_id,  # id of the model repository from the Hugging Face Hub (repo_id = {organization}/{repo_name} for instance ThomasSimonini/ppo-LunarLander-v2
    commit_message=commit_message,
)